In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm 
import wandb
from torch.optim.lr_scheduler import CosineAnnealingLR
import sys
from torch.amp import autocast, GradScaler

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))
from src.utils.k_folds_creator import KFoldCreator
from src.utils.utils import get_models_save_path, set_seeds
from src.utils.constants import Constants 
from src.datasets.eeg_dataset_montage import EEGDatasetMontage
from src.models.gru_convolution_attention import NodeAttentionModel

wandb.login()

2025-11-29 09:24:28,519 :: root :: INFO :: Initialising Utils
2025-11-29 09:24:28,905 :: root :: INFO :: Initialising Datasets
2025-11-29 09:24:28,913 :: root :: INFO :: Initialising Models


Skipping module cbramod_dataset due to missing dependency: No module named 'mne'
Skipping module cbramod_model due to missing dependency: No module named 'einops'


wandb: Currently logged in as: maikotrede (hms-hslu-aicomp-hs25) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
class CFG:
    seed = 42
    n_splits = 5
    data_path = '../../../data/'

    model_name = 'GRUConvNodeAttentionModel'
    hidden_units = 256
    num_layers = 1
    target_size = 6 
    
    num_cnn_blocks = 4 
    
    sampling_rate = 200 # Hz
    sequence_duration = 50 
    downsample_factor = 1
    
    num_channels = 19
    
    dropout = 0.4
    batch_size = 32
    num_workers = 8
    epochs = 50
    lr = 1e-4
    patience = 10
    min_delta = 0.001
    
    use_attention = True

    use_mixup = True
    mixup_alpha = 0.5      # beta distribution parameter (1.0 = uniform distribution between 0 and 1)
    
    

CFG.sequence_length = CFG.sequence_duration * CFG.sampling_rate 

set_seeds(CFG.seed)

TARGETS = Constants.TARGETS

In [3]:
def mixup_data(x, y, alpha=1.0, device='cuda'):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(device)

    mixed_x = lam * x + (1 - lam) * x[index, :]

    mixed_y = lam * y + (1 - lam) * y[index, :]
    
    return mixed_x, mixed_y

In [4]:
def get_dataloaders(df, fold_id):
    train_df = df[df['fold'] != fold_id].reset_index(drop=True)
    valid_df = df[df['fold'] == fold_id].reset_index(drop=True)

    train_dataset = EEGDatasetMontage(df=train_df, data_path=CFG.data_path, mode='train', downsample_factor=CFG.downsample_factor, augmentations=["channel_mask"])

    valid_dataset = EEGDatasetMontage(df=valid_df, data_path=CFG.data_path, mode='valid', downsample_factor=CFG.downsample_factor)

    train_loader = DataLoader(
        train_dataset, batch_size=CFG.batch_size, shuffle=True,
        num_workers=CFG.num_workers, pin_memory=True, drop_last=True, persistent_workers=True if CFG.num_workers > 0 else False
    )
    
    valid_loader = DataLoader(
        valid_dataset, batch_size=CFG.batch_size, shuffle=False,
        num_workers=CFG.num_workers, pin_memory=True, drop_last=False, persistent_workers=True if CFG.num_workers > 0 else False
    )
    
    return train_loader, valid_loader

In [5]:
def run_training(df, DATA_PREPARATION_VOTE_METHOD):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    autocast_enabled = (device.type == 'cuda')
    
    all_oof = []
    all_true = []
    
    fold_scores = []

    for fold in range(CFG.n_splits):
        print(f"\n========== FOLD {fold} ==========")

        experiment_group = f"montages_block_{CFG.num_cnn_blocks}_attention_{CFG.use_attention}_attentionpool4headsMP_RESNET_AUG(MU+CM))"
        experiment_name = f"{experiment_group}_fold{fold}"

        config = {
            "architecture": CFG.model_name, "hidden_units": CFG.hidden_units, 
            "num_layers": CFG.num_layers,
            "fold": fold, "features": "raw_eeg", 
            "sequence_duration": f"{CFG.sequence_duration}s",
            "optimizer": "AdamW", "learning_rate": CFG.lr, 
            "batch_size": CFG.batch_size,
            "epochs": CFG.epochs, "seed": CFG.seed, 
            "Scheduler": "CosineAnnealingLR",
            
            "num_cnn_blocks": CFG.num_cnn_blocks 
        }

        wandb.init(
            project="hms-aicomp-gru-conv",
            name=experiment_name,  
            group=experiment_group,
            tags=['gru-conv', f'fold{fold}', f'montages_block_{CFG.num_cnn_blocks}_attention_{CFG.use_attention}'],
            config=config
        )

        model = NodeAttentionModel(
            num_nodes=CFG.num_channels,       
            node_embed_size=256,              
            hidden_size=CFG.hidden_units,    
            num_layers=CFG.num_layers,       
            num_classes=CFG.target_size,     
            num_cnn_blocks=CFG.num_cnn_blocks,
            dropout=CFG.dropout,
            use_inception=True              
        )
        model.to(device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=1e-2)
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.epochs)
        loss_fn = nn.KLDivLoss(reduction='batchmean')
        train_loader, valid_loader = get_dataloaders(df, fold)

        scaler = GradScaler(enabled=autocast_enabled)

        best_val_loss = float('inf')
        best_model_path = None
        patience_counter = 0
        
        for epoch in range(CFG.epochs):
            print(f"   --- Epoch {epoch+1}/{CFG.epochs} ---")
            
            model.train()
            train_loss = 0
            for signals, labels in tqdm(train_loader, desc="Training"):
                signals, labels = signals.to(device), labels.to(device)
                if CFG.use_mixup:
                    signals, labels = mixup_data(signals, labels, alpha=CFG.mixup_alpha, device=device)
                optimizer.zero_grad()

                with autocast(enabled=autocast_enabled, device_type=device.type):
                    outputs = model(signals)
                    log_probs = F.log_softmax(outputs, dim=1)
                    loss = loss_fn(log_probs, labels)

                scaler.scale(loss).backward()

                scaler.unscale_(optimizer)

                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                scaler.step(optimizer)
                scaler.update()

                train_loss += loss.item() * signals.size(0)
                wandb.log({"train/loss": loss.item()})
            train_loss /= len(train_loader.dataset)

            model.eval()
            valid_loss = 0
            with torch.no_grad():
                for i, (signals, labels) in enumerate(tqdm(valid_loader, desc="Validation")):
                    signals, labels = signals.to(device), labels.to(device)
                    
                    with autocast(enabled=autocast_enabled, device_type=device.type):
                        outputs = model(signals)
                        log_probs = F.log_softmax(outputs, dim=1)
                        loss = loss_fn(log_probs, labels)
                        
                    valid_loss += loss.item() * signals.size(0)

            valid_loss /= len(valid_loader.dataset)
            
            epoch_lr = optimizer.param_groups[0]['lr']
            print(f"   Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Valid Loss = {valid_loss:.4f}, LR = {epoch_lr:.6f}")
            wandb.log({
                "epoch": epoch + 1, "train/epoch_loss": train_loss, "val/loss": valid_loss,
                "val/kl_div": valid_loss, "train/epoch_lr": epoch_lr
            })

            if valid_loss < best_val_loss - CFG.min_delta:
                best_val_loss = valid_loss
                patience_counter = 0 
                
                best_model_path = get_models_save_path() / "GRUConvNodeAttentionModel" / DATA_PREPARATION_VOTE_METHOD / f'best_model_fold{fold}.pth'
                best_model_path.parent.mkdir(parents=True, exist_ok=True)
                torch.save(model.state_dict(), best_model_path)
                print(f"   New best model saved with validation loss: {best_val_loss:.4f}")

            else:
                patience_counter += 1
                print(f"   No improvement. Patience: {patience_counter}/{CFG.patience}")

            if patience_counter >= CFG.patience:
                print(f"   Early stopping triggered after {patience_counter} epochs without improvement.")
                wandb.log({"early_stopped_epoch": epoch + 1})
                break 
            scheduler.step()
        
        fold_scores.append(best_val_loss)
        wandb.summary['best_val_kl_div'] = best_val_loss

        if best_model_path:
            print(f"Loading best model from fold {fold} to generate OOF predictions...")
            model.load_state_dict(torch.load(best_model_path))
            model.eval()

            fold_oof_preds = []
            fold_oof_labels = []
            with torch.no_grad():
                for signals, labels in tqdm(valid_loader, desc=f"Generating OOF for Fold {fold}"):
                    signals = signals.to(device)
                    
                    with autocast(enabled=autocast_enabled, device_type=device.type):
                        outputs = model(signals)
                        probs = F.softmax(outputs, dim=1).cpu().numpy()
                        
                    fold_oof_preds.append(probs)
                    fold_oof_labels.append(labels.cpu().numpy())
            
            all_oof.extend(np.concatenate(fold_oof_preds))
            all_true.extend(np.concatenate(fold_oof_labels))

            artifact = wandb.Artifact(f'model-fold{fold}', type='model')
            artifact.add_file(best_model_path)
            wandb.log_artifact(artifact)
            print(f"\nLogged artifact for fold {fold} with best validation loss: {best_val_loss:.4f}")
        else:
            print("\nNo best model was saved during training for this fold.")
        
        wandb.finish()

    print("\nCalculating final OOF CV score from all collected predictions...")
    
    valid_indices = df[df['fold'].isin(range(CFG.n_splits))].index
    oof_df = pd.DataFrame(all_oof, index=valid_indices, columns=TARGETS)
    true_df = pd.DataFrame(all_true, index=valid_indices, columns=TARGETS)
    
    oof_df = oof_df.sort_index()
    true_df = true_df.sort_index()

    oof_tensor = torch.tensor(oof_df.values, dtype=torch.float32)
    true_tensor = torch.tensor(true_df.values, dtype=torch.float32)

    kl_loss = nn.KLDivLoss(reduction="batchmean")
    overall_cv_score = kl_loss(torch.log(oof_tensor), true_tensor).item()

    return overall_cv_score, fold_scores

DATA_PREPARATION_VOTE_METHOD = "max_vote_window"

print("Preparing data and creating folds...")
df = pd.read_csv(CFG.data_path + 'processed_data_max_vote_window.csv') 

label_map = {t: i for i, t in enumerate(TARGETS)}
df['expert_consensus'] = df[TARGETS].idxmax(axis=1)

print('Train shape:', df.shape)
print('Targets', list(TARGETS))

fold_creator = KFoldCreator(n_splits=CFG.n_splits, seed=CFG.seed)
df = fold_creator.create_folds(df, stratify_col='expert_consensus', group_col='patient_id')

print("Folds created. Value counts per fold:")
print(df['fold'].value_counts())

Preparing data and creating folds...
Train shape: (17089, 15)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
Folds created. Value counts per fold:
fold
0    4067
1    3658
2    3381
4    3358
3    2625
Name: count, dtype: int64


In [6]:
experiments_to_run = [3]  

experiment_results = {}

print(f"Starting systematic evaluation for num_cnn_blocks: {experiments_to_run}...")

for num_blocks in experiments_to_run:
    print("\n" + "="*60)
    print(f"      STARTING EXPERIMENT: {num_blocks} CNN BLOCKS")
    print("="*60)

    CFG.num_cnn_blocks = num_blocks

    overall_cv_score, all_fold_scores = run_training(df, DATA_PREPARATION_VOTE_METHOD)
    
    mean_fold_score = np.mean(all_fold_scores)
    
    experiment_results[num_blocks] = {
        'overall_cv_score': overall_cv_score,
        'mean_fold_score': mean_fold_score
    }
    
    print("\n" + "="*60)
    print(f"      FINISHED EXPERIMENT: {num_blocks} CNN BLOCKS")
    print(f"      OOF KL Score (all folds): {overall_cv_score:.4f}")
    print(f"      Mean of fold scores: {mean_fold_score:.4f}")
    print("="*60)


for num_blocks, results in experiment_results.items():
    print(f"{num_blocks:<12} | {results['overall_cv_score']:<18.4f} | {results['mean_fold_score']:<18.4f}")

print("="*50)
print("All experiments complete. Check wandb for detailed charts.")

Starting systematic evaluation for num_cnn_blocks: [3]...

      STARTING EXPERIMENT: 3 CNN BLOCKS
Using device: cuda

========== FOLD 0 ==========


   --- Epoch 1/50 ---


Training:   0%|          | 0/406 [00:00<?, ?it/s]

Validation:   0%|          | 0/128 [00:00<?, ?it/s]

   Epoch 1: Train Loss = 0.9163, Valid Loss = 1.0686, LR = 0.000100
   New best model saved with validation loss: 1.0686
   --- Epoch 2/50 ---


...

   Epoch 42: Train Loss = 0.4099, Valid Loss = 0.5767, LR = 0.000008
   No improvement. Patience: 10/10
   Early stopping triggered after 10 epochs without improvement.
Loading best model from fold 4 to generate OOF predictions...


Generating OOF for Fold 4:   0%|          | 0/105 [00:00<?, ?it/s]


Logged artifact for fold 4 with best validation loss: 0.5764


early_stopped_epoch,▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
train/epoch_loss,█▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch_lr,█████████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,▆▅█▄▄▂▆▆▂▄▅▁▁▂▂▂▃▃▃▃▃▄▃▅▄▂▂▃▂▁▂▁▄▁▂▁▂▁▂▁
val/kl_div,█▄▃▄▂▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/loss,█▄▃▄▂▃▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_val_kl_div,0.57639
early_stopped_epoch,42
epoch,42
train/epoch_loss,0.40988



Calculating final OOF CV score from all collected predictions...

      FINISHED EXPERIMENT: 3 CNN BLOCKS
      OOF KL Score (all folds): 0.5676
      Mean of fold scores: 0.5661
3            | 0.5676             | 0.5661            
All experiments complete. Check wandb for detailed charts.


In [7]:
print(f"OOF KL Score (calculated across all folds): {overall_cv_score:.4f}")
print(f"Mean of individual fold scores: {np.mean(all_fold_scores):.4f}")

OOF KL Score (calculated across all folds): 0.5676
Mean of individual fold scores: 0.5661
